### Required library

In [21]:
import os
from dotenv import load_dotenv
from coinbase.rest import RESTClient
from json import dumps
from datetime import datetime, timedelta
import pandas as pd
import uuid


### 1) Provide API key

In [22]:
load_dotenv()
api_key = os.getenv("api_key")
api_secret = os.getenv("api_secret")

# Create Instance
client = RESTClient(api_key=api_key, api_secret=api_secret)

### 2) Get account information

In [23]:
accounts = client.get_accounts()
# print(dumps(accounts, indent=2))

### 3) Get specific account information

In [25]:
account_uuid = os.getenv("account_uuid")
account_details = client.get_account(account_uuid)
# print(dumps(account_details, indent=2))

### 4) Get Time

In [31]:
time_dict = client.get_unix_time()
iso_time = time_dict["iso"]
epoch_seconds = int(time_dict["epochSeconds"])
epoch_millis = int(time_dict["epochMillis"])

print(f"Time_dict : {time_dict}")
print(f"ISO Time : {iso_time}")
print(f"Epoch Seconds : {epoch_seconds}")
print(f"Epoch Millis : {epoch_millis}")

Time_dict : {'iso': '2024-09-18T14:31:16Z', 'epochSeconds': '1726669876', 'epochMillis': '1726669876281'}
ISO Time : 2024-09-18T14:31:16Z
Epoch Seconds : 1726669876
Epoch Millis : 1726669876281


### 5) Retrieve Fees Charged

In [33]:
product_type = "SPOT"
contract_expiry_type = "UNKNOW_CONTRACT"
summary = client.get_transaction_summary()
print(type(summary))
print(summary)

<class 'dict'>
{'total_volume': 0, 'total_fees': 0, 'fee_tier': {'pricing_tier': 'Advanced 1', 'usd_from': '0', 'usd_to': '1000', 'taker_fee_rate': '0.012', 'maker_fee_rate': '0.006', 'aop_from': '', 'aop_to': ''}, 'margin_rate': None, 'goods_and_services_tax': None, 'advanced_trade_only_volume': 0, 'advanced_trade_only_fees': 0, 'coinbase_pro_volume': 0, 'coinbase_pro_fees': 0, 'total_balance': '10.00', 'has_promo_fee': False}


### 6) Get Market Data

In [53]:
product_id = "BTC-USD"

# 86400 = 24 x 60 x 60
end = datetime.now()
start = end - timedelta(hours=24)

end_unix = int(end.timestamp())
start_unix = int(start.timestamp())

# ONE_MINUTE, FIVE_MINUTE, FIFTEEN_MINUTE, THIRTY_MINUTE, ONE_HOUR, TWO_HOUR, SIX_HOUR, ONE_DATE
granularity = "ONE_HOUR"

data = client.get_candles(product_id, start_unix, end_unix, granularity)
candle_data = data["candles"]

# unit = (D,s,ms,us,ns)
df_candles = pd.DataFrame(candle_data)
df_candles['start'] = pd.to_datetime(df['start'], unit='s')

df_candles.head()


,start,low,high,open,close,volume
0,2024-09-18 14:00:32,59326.22,59771.37,59496.64,59537.51,591.24751032
1,2024-09-18 13:00:48,59285.17,60166.29,59981.5,59496.64,589.57892615
2,2024-09-18 12:01:04,59689.69,60005.53,59848.5,59981.5,198.14410231
3,2024-09-18 10:59:12,59825,60017.25,59907.58,59848.5,107.4392174
4,2024-09-18 09:59:28,59382.17,59927.4,59661.87,59907.58,335.88343756


### 7) Get Trade Data

In [52]:
product_id = "BTC-USD"
limit = "50"
data = client.get_market_trades(product_id, limit)
trade_data = data["trades"]

df_trade = pd.DataFrame(trade_data)
df_trade.head()


,trade_id,product_id,price,size,time,side,bid,ask
0,693257865,BTC-USD,59528.82,0.00003,2024-09-18T14:56:08.336568Z,BUY,,
1,693257864,BTC-USD,59523.65,0.00003,2024-09-18T14:56:08.292235Z,SELL,,
2,693257863,BTC-USD,59523.65,0.00002148,2024-09-18T14:56:08.292235Z,SELL,,
3,693257862,BTC-USD,59523.65,0.00026166,2024-09-18T14:56:08.292235Z,SELL,,
4,693257861,BTC-USD,59525.77,0.00135269,2024-09-18T14:56:08.022790Z,SELL,,
